## Задание к лекции "Основы веб-скрапинга и работы с API"
### Задание 1
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
from datetime import datetime

In [15]:
KEYWORDS = ['python', 'парсинг']
def get_results (words, URL_1 = 'https://habr.com/ru/all/'):
    res = requests.get(URL_1)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('article', class_ = 'post post_preview')
    
    preview = pd.DataFrame()
    
    for news_block in news_blocks:
        for key in KEYWORDS:
            if key.lower() in news_block.text.lower():
                date = news_block.find('span', class_ = 'post__time').text
                title = news_block.find('h2', class_ = 'post__title').text
                link = news_block.find('a', class_ = 'post__title_link').get('href')
                all_inf = {'date': date, 'title': title.strip(), 'link': link}
                preview = pd.concat([preview, pd.DataFrame([all_inf])])
          
    preview = preview.drop_duplicates()
    return preview
get_results (KEYWORDS)

,date,title,link
0,сегодня в 13:56,С каких книг можно начать изучать программиров...,https://habr.com/ru/post/536172/
0,сегодня в 13:13,Как я научила свой компьютер играть в пары исп...,https://habr.com/ru/post/536162/


## Задание 2.

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [14]:
EMAIL = ['metallurg_annet@mail.ru']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
addresses = {'emailAddresses': EMAIL}

res = requests.post(url, json=addresses, headers=headers)

df = pd.DataFrame(res.json()['breaches']).T.reset_index() 

df = df[['site', 'description', 'publishDate']]
df['email'] = [', '.join(el.keys()) for el in list(res.json()['data'].values())]

df

,site,description,publishDate,email
0,twitter.com,Login credentials for over 32 Million Twitter ...,2016-10-26T00:00:00Z,metallurg_annet@mail.ru
1,eina.ru,"In November 2020, a collection of over 23,000 ...",2020-12-10T00:00:00Z,metallurg_annet@mail.ru
2,last.fm,"In March 2012, the British music streaming ser...",2016-10-25T00:00:00Z,metallurg_annet@mail.ru
3,adobe.com,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z,metallurg_annet@mail.ru
4,livejournal.com,"In 2017, social network LiveJournal's database...",2019-05-23T00:00:00Z,metallurg_annet@mail.ru
